In [ ]:
import copy
import codecs

import sacremoses
import conllu

from rule import Question, AnswerSpan

In [ ]:
detokenizer = sacremoses.MosesDetokenizer()

## Parse conllu file

In [ ]:
print('Parsing conllu file...')
with codecs.open('examples.conllu', 'r', encoding='utf-8') as f:
    conllu_file = conllu.parse(f.read())

In [ ]:
# Creating dict
ids = range(int(len(conllu_file)/2))
examples = {}
count = 0
for i, s in enumerate(conllu_file):
    if i % 2 == 0:
        examples[ids[count]] = s
    else:
        examples[str(ids[count])+'_answer'] = s
        count +=1

## Perform transformation

In [ ]:
def qa2d(idx):
    q = Question(copy.deepcopy(examples[idx].tokens))
    if not q.isvalid:
        print("Question {} is not valid.".format(idx))
        return ''
    a = AnswerSpan(copy.deepcopy(examples[str(idx)+'_answer'].tokens))
    if not a.isvalid:
        print("Answer span {} is not valid.".format(idx))
        return ''
    q.insert_answer_default(a)
    return detokenizer.detokenize(q.format_declr(), return_str=True)

In [ ]:
def print_sentence(idx):
    return detokenizer.detokenize([examples[idx].tokens[i]['form'] for i in range(len(examples[idx].tokens))], return_str=True)

In [ ]:
total = int(len(examples.keys())/2)
print("Transforming {} examples.".format(total))
for i in range(total):
    out = qa2d(i)
    print(print_sentence(i))
    if out != '':
        print(out)
    print('----------')
 